# Bedrock AgentCore Runtime에서 MCP 서버 배포

이 워크샵은 Amazon Bedrock AgentCore Runtime을 사용하여 Model Context Protocol (MCP) 서버를 배포하고 사용하는 방법을 보여주며, 

AI 에이전트를 위한 사용자 정의 도구의 확장 가능하고 안전한 배포를 가능하게 합니다.

## 개요

이 실습에서는 다음을 수행합니다:
- 웹 검색 기능을 가진 사용자 정의 MCP 서버 생성
- Amazon Cognito를 사용한 인증 설정
- Bedrock AgentCore Runtime에 MCP 서버 배포
- Strands Agents로 배포된 서버 테스트

## 사전 요구사항

이 실습을 시작하기 전에 다음이 준비되어 있는지 확인하세요:
- AWS 자격 증명 구성 (IAM 역할 또는 환경 변수)
- 필요한 Python 패키지 설치
- AWS 리전에 따른 Nova Pro 모델 ID

IAM 역할이 가정된 환경에서 실행하지 않는 경우, AWS 자격 증명을 환경 변수로 설정하세요:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

MCP 서버 개발, Strands Agents 및 Bedrock AgentCore SDK에 필요한 패키지를 설치하세요:

In [ ]:
#%pip install -q ddgs mcp strands-agents strands-agents-tools bedrock-agentcore bedrock-agentcore-starter-toolkit rich

AWS 리전에 따라 Nova Pro 모델 ID를 설정하세요:

In [ ]:
import boto3

region = boto3.session.Session().region_name

NOVA_PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
if region.startswith("eu"):
    NOVA_PRO_MODEL_ID = "eu.amazon.nova-pro-v1:0"
elif region.startswith("ap"):
    NOVA_PRO_MODEL_ID = "apac.amazon.nova-pro-v1:0"

print(f"Nova Pro Model ID: {NOVA_PRO_MODEL_ID}")

## MCP용 Bedrock AgentCore Runtime이란 무엇인가요?

Amazon Bedrock AgentCore Runtime을 사용하면 Model Context Protocol (MCP) 서버를 관리형 확장 가능한 서비스로 배포할 수 있습니다. 주요 이점은 다음과 같습니다:

- **확장성**: 수요에 따라 자동으로 확장
- **보안**: 내장된 인증 및 권한 부여
- **관리형 인프라**: 서버나 컨테이너를 관리할 필요 없음
- **통합**: Bedrock 서비스와의 원활한 통합

MCP 서버는 AI 에이전트가 웹 검색, 데이터베이스 액세스 또는 사용자 정의 비즈니스 로직과 같은 기능을 확장하는 데 사용할 수 있는 도구와 리소스를 제공합니다.

### 사용자 정의 MCP 서버 생성

DuckDuckGo (개인정보 보호를 중시하는 검색 엔진) 를 사용하여 웹 검색 기능을 제공하는 간단한 MCP 서버를 생성해보겠습니다. 

이 서버는 확장 가능한 사용을 위해 AgentCore Runtime에 배포됩니다.

In [ ]:
# 나중에 사용을 위해 mcp_server.py 파일을 생성합니다.
# %%writefile mcp_server.py

# mcp_server.py 파일 생성
mcp_server_code = '''from mcp.server.fastmcp import FastMCP
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

# 웹 검색 도구 정의
@mcp.tool()
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> list:
    """최신 정보를 얻기 위해 웹을 검색합니다.
    Args:
        keywords (str): 검색 쿼리 키워드.
        region (str): 검색 지역: wt-wt, us-en, uk-en, ru-ru 등.
        max_results (int | None): 반환할 최대 결과 수.
    Returns:
        검색 결과가 포함된 딕셔너리 목록.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "결과를 찾을 수 없습니다."
    except RatelimitException:
        return "RatelimitException: 잠시 후 다시 시도해주세요."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

if __name__ == "__main__":
    mcp.run(transport="streamable-http")'''

with open('mcp_server.py', 'w', encoding='utf-8') as f:
    f.write(mcp_server_code)

print("mcp_server.py 파일이 생성되었습니다.")


In [ ]:
# %%writefile requirements.txt
req_code = '''ddgs
mcp
bedrock-agentcore'''

with open('requirements.txt', 'w', encoding='utf-8') as f:
    f.write(req_code)

print("requirements.txt 파일이 생성되었습니다.")

### MCP 서버 로컬 테스트 (선택사항 - 워크샵 환경에서 포트 8000이 사용 중인 경우 건너뛰기)

AgentCore Runtime에 배포하기 전에 MCP 서버를 로컬에서 테스트합니다.

### 1단계: MCP 서버 시작

터미널에서 MCP 서버를 실행하세요:

```bash
cd 04-bedrock-agentcore-runtime-mcp/
uv pip install -r requirements.txt
uv run mcp_server.py
```
또는
```bash
cd 04-bedrock-agentcore-runtime-mcp/
pip install -r requirements.txt
python mcp_server.py
```

### 2단계: Strands Agent로 테스트

통합을 테스트하기 위해 다음 코드를 실행하세요:

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

# 웹 검색 MCP 서버에 연결
print("\nMCP 서버에 연결 중...")
mcp_url = f"http://localhost:8000/mcp"
websearch_server = MCPClient(lambda: streamablehttp_client(mcp_url))

with websearch_server:
    mcp_tools = (websearch_server.list_tools_sync())
    print(f"사용 가능한 MCP 도구: {[tool.tool_name for tool in mcp_tools]}")

    # 자체 구축한 MCP 도구로 에이전트 생성
    agent = Agent(
        model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
        system_prompt = """당신은 간결한 응답을 제공하는 도움이 되는 어시스턴트입니다.
                        웹 검색이 필요한 경우, 항상 websearch 도구를 먼저 사용하세요.
                        """,
        tools=mcp_tools,
    )

    agent("Bedrock AgentCore란 무엇인가요?")

### 로컬에서 실행 중인 MCP 서버 중지

MCP 서버를 로컬에서 테스트한 후, 터미널에서 `Ctrl+C`를 눌러 로컬에서 실행 중인 MCP 서버를 중지하세요.

## 인증을 사용하여 Bedrock AgentCore Runtime에 MCP 서버 배포
이제 MCP 서버를 Bedrock AgentCore Runtime에 구성하고 배포하겠습니다. 이 과정에는 종속성 생성, 인증 구성 및 서비스 배포가 포함됩니다.
![bedrock-agentcore-runtime-launch](images/runtime-launch.png)

### 1단계: 인바운드 인증을 위한 Amazon Cognito 설정

배포된 MCP 서버에 대한 보안 액세스를 위해 Cognito 사용자 풀을 생성합니다.

생성된 구성 요소
- **사용자 풀**: 사용자 ID 관리
- **앱 클라이언트**: 애플리케이션 인증 활성화
- **테스트 사용자**: 인증 흐름 테스트용

In [ ]:
import boto3

region = boto3.session.Session().region_name

# Cognito 클라이언트 초기화
cognito_client = boto3.client('cognito-idp', region_name=region)

# 사용자 풀 생성
user_pool_response = cognito_client.create_user_pool(
    PoolName='MCPServerPool',
    Policies={
        'PasswordPolicy': {
            'MinimumLength': 8
        }
    }
)
cognito_pool_id = user_pool_response['UserPool']['Id']

# 앱 클라이언트 생성
app_client_response = cognito_client.create_user_pool_client(
    UserPoolId=cognito_pool_id,
    ClientName='MCPServerPoolClient',
    GenerateSecret=False,
    ExplicitAuthFlows=[
        'ALLOW_USER_PASSWORD_AUTH',
        'ALLOW_REFRESH_TOKEN_AUTH'
    ]
)
cognito_client_id = app_client_response['UserPoolClient']['ClientId']

# 임시 비밀번호로 사용자 생성
cognito_client.admin_create_user(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    TemporaryPassword='Temp123!',
    MessageAction='SUPPRESS'
)

# 영구 비밀번호 설정
cognito_client.admin_set_user_password(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    Password='MyPassword123!',
    Permanent=True
)

# 필요한 값들 출력
print(f"풀 ID: {cognito_pool_id}")
print(f"디스커버리 URL: https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration")
print(f"클라이언트 ID: {cognito_client_id}")

### 2단계: Bedrock AgentCore Runtime 구성

자동 리소스 생성을 통해 Bedrock AgentCore Runtime 구성을 설정합니다.

**생성된 아티팩트:**
이 단계에서는 필수 배포 파일을 생성합니다:
- **Dockerfile**: MCP 서버용 컨테이너 구성
- **.dockerignore**: docker build 시 제외할 파일 목록
- **.bedrock_agentcore.yaml**: 런타임 배포 구성

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import boto3

region = boto3.session.Session().region_name
print(f"AWS 리전 사용: {region}")

agentcore_runtime = Runtime()

print("AgentCore Runtime 구성 중...")
# agentcore runtime 에 대해 configure 메소드를 호출하면 Dockerfile 과 메타데이터 설정파일 (.bedrock_agentcore.yaml) 생성합니다.
# agentcore runtime 에 대해 configure 를 호출할 때 인가자 (authorize) 로써 cognito 를 설정합니다.
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    protocol="MCP",
    agent_name="mcp_server_agentcore",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_client_id],
            "discoveryUrl": f"https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration",
        }
    }
)
print("구성 완료 ✓")

### 3단계: Bedrock AgentCore Runtime에 배포

컨테이너화 및 배포를 위해 AWS CodeBuild를 사용하여 배포 프로세스를 시작합니다.

**배포 프로세스:**
- MCP 서버의 컨테이너화된 버전 빌드
- 필요한 AWS 리소스 생성 (ECR 리포지토리, IAM 역할)
- Amazon ECR에 컨테이너 이미지 푸시
- 관리형 자동 확장 서비스로 AgentCore Runtime에 배포

In [ ]:

# agentcore runtime 에 대해 launch 메소드를 호출하면 실제로 container 환경에 agent 를 배포합니다.

launch_result = agentcore_runtime.launch()

### 배포 상태 확인

배포 상태를 확인하고 런타임이 준비될 때까지 기다립니다:

In [ ]:
import time

print("AgentCore Runtime 상태 확인 중...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"초기 상태: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"상태: {status} - 대기 중...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime이 준비되었습니다!")
else:
    print(f"⚠ AgentCore Runtime 상태: {status}")
    
print(f"최종 상태: {status}")

mcp_runtime_id = launch_result.agent_id
mcp_runtime_arn = launch_result.agent_arn
ecr_repo_name = launch_result.ecr_uri.split('/')[1]
codebuild_name = launch_result.codebuild_id.split(':')[0]
print(f"MCP AgentCore Runtime ID: {mcp_runtime_id}")
print(f"MCP AgentCore Runtime ARN: {mcp_runtime_arn}")
print(f"MCP AgentCore Runtime용 ECR 리포지토리: {ecr_repo_name}")
print(f"Strands AgentCore Runtime용 CodeBuild 프로젝트: {codebuild_name}")

### Strands Agent와 함께 배포된 MCP 서버를 도구로 테스트

이제 적절한 인증을 통해 Bedrock AgentCore Runtime 엔드포인트에 연결하여 배포된 MCP 서버를 테스트해보겠습니다.

먼저 사용자 이름과 비밀번호로 Cognito 인증에서 액세스 토큰을 얻습니다.

In [ ]:
import boto3

region = boto3.session.Session().region_name

# Cognito 인증에서 베어러 토큰(액세스 토큰) 가져오기
cognito_client = boto3.client('cognito-idp', region_name=boto3.session.Session().region_name)
auth_response = cognito_client.initiate_auth(
    ClientId=cognito_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': 'testuser',
        'PASSWORD': 'MyPassword123!'
    }
)
bearer_token = auth_response['AuthenticationResult']['AccessToken']
print(bearer_token)

그런 다음 액세스 토큰을 요청 헤더의 베어러로 설정하여 AgentCore Runtime에서 호스팅되는 MCP 서버에 안전하게 연결합니다.

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

region = boto3.session.Session().region_name

mcp_runtime_arn = launch_result.agent_arn
encoded_arn = mcp_runtime_arn.replace(':', '%3A').replace('/', '%2F')

# 웹 검색 MCP 서버에 연결
print("\nMCP 서버에 연결 중...")
mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
headers = {
    "Authorization": f"Bearer {bearer_token}",
    #"Content-Type": "application/json"
}
websearch_server = MCPClient(lambda: streamablehttp_client(mcp_url, headers, terminate_on_close=False))

# 아래에서 agent 는 mcp tool 리스트를 얻을 때 mcp server 에 연결하여 tool 리스트를 획득합니다.
with websearch_server:
    mcp_tools = (websearch_server.list_tools_sync())
    print(f"사용 가능한 도구: {[tool.tool_name for tool in mcp_tools]}")

    # 자체 구축한 MCP 도구로 에이전트 생성
    agent = Agent(
        model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
        system_prompt = """당신은 간결한 응답을 제공하는 도움이 되는 어시스턴트입니다.
                        웹 검색이 필요한 경우, 항상 websearch 도구를 먼저 사용하세요.
                        """,
        tools=mcp_tools,
    )

    agent("Bedrock AgentCore란 무엇인가요?")


### console 에서 확인

AWS Console 의 [AgentCore Runtime](https://us-west-2.console.aws.amazon.com/bedrock-agentcore/agents) 에서 배포된 runtime 을 확인할 수 있습니다.

에이전트가 요청을 처리하고 응답을 생성하는 방법을 이해하기 위해 에이전트 루프의 상세한 실행 흐름을 살펴보겠습니다:

In [ ]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("에이전트 루프 세부사항")
console.rule()
console.print(f"루프 수: {agent.event_loop_metrics.cycle_count}")

table = Table(title="에이전트 메시지", show_lines=True)
table.add_column("역할", style="green")
table.add_column("텍스트", style="magenta")
table.add_column("도구 이름", style="cyan")
table.add_column("도구 입력", style="cyan")
table.add_column("도구 결과", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content]
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

## 리소스 정리 (선택사항)

배포된 리소스를 정리합니다:

In [ ]:
import boto3
import os

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr',region_name=region)
codebuild_client = boto3.client('codebuild',region_name=region)
cognito_client = boto3.client('cognito-idp', region_name=region)

try:
    print("AgentCore Runtime 삭제 중...")
    agentcore_control_client.delete_agent_runtime(agentRuntimeId=mcp_runtime_id)
    print("✓ AgentCore Runtime 삭제가 시작되었습니다")

    print("ECR 리포지토리 삭제 중...")
    ecr_client.delete_repository(repositoryName=ecr_repo_name, force=True)
    print("✓ ECR 리포지토리가 삭제되었습니다")

    print("CodeBuild 프로젝트 삭제 중...")
    codebuild_client.delete_project(name=codebuild_name)
    print("✓ CodeBuild 프로젝트가 삭제되었습니다")

    print("Cognito 사용자 풀 삭제 중...")
    cognito_client.delete_user_pool(UserPoolId=cognito_pool_id)
    print("✓ Cognito 사용자 풀이 삭제되었습니다")

    print("Bedrock AgentCore 구성 파일 삭제 중...")
    os.remove(".bedrock_agentcore.yaml") 
    print("✓ .bedrock_agentcore.yaml이 삭제되었습니다")
except Exception as e:
    print(f"❌ 정리 중 오류 발생: {e}")
    print("일부 리소스를 수동으로 정리해야 할 수 있습니다.")

## 결론

이 실습에서 성공적으로 다음을 수행했습니다:

- ✅ DuckDuckGo를 사용하여 웹 검색 기능을 가진 사용자 정의 MCP 서버 생성
- ✅ MCP 서버에 대한 보안 인증을 위해 Amazon Cognito 설정
- ✅ Bedrock AgentCore Runtime에 MCP 서버 구성 및 배포
- ✅ AI 기반 워크플로를 위해 배포된 MCP 서버를 Strands Agents와 통합
  
## MCP용 AgentCore Runtime의 주요 이점

- **확장 가능한 배포**: 서버 관리 없이 MCP 서버를 위한 관리형 인프라
- **보안 인증**: Cognito 및 기타 인증 방법에 대한 내장 지원
- **쉬운 통합**: Strands Agents 및 기타 AI 프레임워크와의 원활한 연결
- **프로덕션 준비**: 엔터프라이즈급 안정성 및 모니터링 기능